# softmax classification
- sigmoid 함수 값이 여러개 생겨서 만들어진 함수를 softmax 함수라고 생각하면됨
- classification 되어 있는 각각의 값을 모두 더한 값이 1이되고 그 중 가장 높은 값을 정답 데이터로 하여 추출해내는 방식

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


###### softmax의 경우 onehot 인코딩을 통해서 정답 데이터 값을 만들어줌(y_data의 경우 3개의 값을 가지고 각각 포함될 경우 1의 값을 얻게 함)

In [2]:
x_data = [[1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5], [1,2,5,6], [1,6,6,6], [1,7,7,7]]
y_data = [[0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], [1,0,0],[1,0,0]]

In [3]:
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

In [4]:
W = tf.Variable(tf.random_normal([4, nb_classes]), name="Weight")
b = tf.Variable(tf.random_normal([nb_classes]), name="bias")

In [5]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

### softmax의 cost 값 해석
- 여러개의 sigmoid함수를 사용하고 있는 함수이므로 cost(loss) 값을 구할 때 모든 hypothesis를 더하여 나타내준 값의 평균이 cost가 됨

In [6]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
for step in range(2001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 200 == 0:
        print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

0 6.4971895
200 0.5702657
400 0.46471596
600 0.38669544
800 0.3141119
1000 0.24607968
1200 0.22071692
1400 0.2010678
1600 0.18453386
1800 0.17043114
2000 0.1582658


# one_hot encoding

In [9]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

###### tf.arg_max는 값 중 제일 큰 값을 출력해줌

In [10]:
a = sess.run(hypothesis, feed_dict={X: [[1,11,7,9]]})
print(a, sess.run(tf.arg_max(a,1)))

[[6.0946802e-03 9.9389511e-01 1.0242175e-05]] [1]


In [11]:
all = sess.run(hypothesis, feed_dict={X: [[1,11,7,9],[1,3,4,3,],[1,1,0,1]]})
print(all, sess.run(tf.arg_max(all,1)))

[[6.0946830e-03 9.9389511e-01 1.0242185e-05]
 [8.4885556e-01 1.3628161e-01 1.4862840e-02]
 [1.6046489e-08 3.7650237e-04 9.9962342e-01]] [1 0 2]


# softmax_cross_entropy_with_logits

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [13]:
x_data = [[1,2,2,3], [1,2,6,1], [1,2,5,6], [4,1,3,2], [4,2,2,5], [4,1,5,5], [1,6,6,6], [1,6,6,6]]
y_data = [[0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], [1,0,0],[1,0,0]]

# 핵심 X 값은 float로 처리하고 Y 값은 onehot 할 경우 int 처리해야함

In [14]:
X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.int32, [None, 3])
nb_classes = 3

###### one_hot(Y,nb_classes)  one_hot encoding //  tf.reshape(Y_one_hot, [-1, nb_classes]) 입력 값은 reshape해주고 output은 정답 데이터 개수 작성

In [15]:
Y_one_hot = tf.one_hot(Y,nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [16]:
W = tf.Variable(tf.random_normal([4, nb_classes]), name="Weight")
b = tf.Variable(tf.random_normal([nb_classes]), name="bias")

In [17]:
logits = tf.matmul(X, W) + b

In [18]:
hypothesis = tf.nn.softmax(logits)

In [19]:
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [20]:
prediction = tf.arg_max(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [22]:
for step in range(2000):
    sess.run(optimizer, feed_dict={X: x_data, Y_one_hot: y_data})
    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y_one_hot: y_data})
        print("step: {:5}\tLoss: {:3f}\tAcc: {:2f}".format(step, loss, acc))

step:     0	Loss: 7.868546	Acc: 0.375000
step:   100	Loss: 0.090684	Acc: 1.000000
step:   200	Loss: 0.061842	Acc: 1.000000
step:   300	Loss: 0.047623	Acc: 1.000000
step:   400	Loss: 0.038814	Acc: 1.000000
step:   500	Loss: 0.032761	Acc: 1.000000
step:   600	Loss: 0.028335	Acc: 1.000000
step:   700	Loss: 0.024954	Acc: 1.000000
step:   800	Loss: 0.022288	Acc: 1.000000
step:   900	Loss: 0.020131	Acc: 1.000000
step:  1000	Loss: 0.018351	Acc: 1.000000
step:  1100	Loss: 0.016858	Acc: 1.000000
step:  1200	Loss: 0.015587	Acc: 1.000000
step:  1300	Loss: 0.014493	Acc: 1.000000
step:  1400	Loss: 0.013541	Acc: 1.000000
step:  1500	Loss: 0.012705	Acc: 1.000000
step:  1600	Loss: 0.011966	Acc: 1.000000
step:  1700	Loss: 0.011307	Acc: 1.000000
step:  1800	Loss: 0.010717	Acc: 1.000000
step:  1900	Loss: 0.010185	Acc: 1.000000


# flatten을 통해서 값 생성이 어려워 코드 변경으로 flatten 효과를 줌

In [24]:
import numpy as np
y_data_flatten = []
for data in y_data:
    data = np.array(data, dtype="int64")
    data = np.argmax(data)
    y_data_flatten.append(data)
y_data_flatten = np.array(y_data_flatten, dtype="int64")

In [26]:
pred = sess.run(prediction, feed_dict={X: x_data})
for p, y in zip (pred, y_data_flatten):
    print("[{}] prediction: {} True Y: {}".format(p == y, p, y))

[True] prediction: 2 True Y: 2
[True] prediction: 2 True Y: 2
[True] prediction: 2 True Y: 2
[True] prediction: 1 True Y: 1
[True] prediction: 1 True Y: 1
[True] prediction: 1 True Y: 1
[True] prediction: 0 True Y: 0
[True] prediction: 0 True Y: 0
